In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import category_encoders as ce
from pycaret.classification import *
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_train = pd.read_csv("../input/hranalysis/train.csv")
df_test = pd.read_csv("../input/hranalysis/test.csv")

In [ ]:
df_train

In [ ]:
df_train.isnull().sum()

In [ ]:
def fillna(df,column,value):
    df[column].fillna(value,inplace=True)
    return df


In [ ]:
df_train = fillna(df_train,"education","unknown")
df_train = fillna(df_train,"previous_year_rating",0.0)
df_train.drop("employee_id",axis=1,inplace=True)


In [ ]:
df_train.isnull().sum()

In [ ]:
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [ ]:
for column in df_train.columns:
    if df_train[column].dtypes=="object" and column!="gender":
        df_train=one_hot_encoding(df_train,column)

In [ ]:
def ordinal_encoding(df,col,mapping):
    ordinal_encoder = ce.OrdinalEncoder(cols = [col],return_df = True,mapping = [{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final

In [ ]:
df_train = ordinal_encoding(df_train,"gender",{"m":1,"f":0})

In [ ]:
df_train.dtypes

In [ ]:
X = df_train.drop("is_promoted",axis=1)
Y = pd.DataFrame(df_train["is_promoted"])

In [ ]:
smote = SMOTE()
X, Y = smote.fit_resample(X, Y)

In [ ]:
df_train = pd.concat([X,Y],axis = 1)

In [ ]:
df_train

In [ ]:
# experiment = setup(df_train, target="is_promoted")

In [ ]:
# compare_models()

In [ ]:
df_test = fillna(df_test,"education","unknown")
df_test = fillna(df_test,"previous_year_rating",0.0)
df_test_copy = df_test.copy()
df_test.drop("employee_id",axis=1,inplace=True)
for column in df_test.columns:
    if df_test[column].dtypes=="object" and column!="gender":
        df_test=one_hot_encoding(df_test,column)
df_test = ordinal_encoding(df_test,"gender",{"m":1,"f":0})

In [ ]:
X_train = normalize(df_train.drop("is_promoted",axis = 1))
X_test = normalize(df_test)
Y_train = df_train["is_promoted"]

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train,Y_train)

In [ ]:
Y_pred = rfc.predict(X_test)

In [ ]:
submission = pd.DataFrame({"employee_id":df_test_copy["employee_id"],"is_promoted":[0 for i in Y_pred]})

In [ ]:
submission.to_csv("submission.csv",index= False)

In [ ]:
nan